# ptt 爬蟲實戰

先載入要使用的套件

In [2]:
import sys
import os
import requests
import json
import time
import re

建立一個簡單的 debug 小函式，可以幫我們把爬下來的東西存成html，用瀏覽器檢視

In [3]:
def save_html(text):
    with open('saved.html', 'w', encoding="utf8") as f:
        f.write(text)

## 如何爬取整個網頁
我寫一個爬蟲函式作為範例，爬取表特版的首頁

In [4]:
url = 'https://www.ptt.cc/bbs/Beauty/'
def crawl_board():
    res = requests.get(url=url)
    print(res)
#     print(res.text)
    save_html(res.text)

crawl_board()

<Response [200]>


## 爬取不同的版
多加一個參數控制要爬取的版

### 小注意
PTT有個已滿18歲的確認頁面，如要避開，請加上這個參數 `cookies={'over18': '1'}` 到 `requests.get`內

In [5]:
base_url = 'https://www.ptt.cc/bbs/'
def crawl_board(board_name):
    url = base_url + board_name
    res = requests.get(url=url)
    # res = requests.get(url=url, cookies={'over18': '1'})

    print(res)
#     print(res.text)
    save_html(res.text)

crawl_board('Gossiping')

<Response [200]>


### 我們現在都只看的到某個版的最後一頁，如果想要看其他頁呢?

這個時候就需要一點人工觀察，例如八卦版的頁面網址長這樣
```
https://www.ptt.cc/bbs/Gossiping/index39241.html
```
分析來看是: base url + 版名 + 版頁數的組成的一些東西

In [6]:
base_url = 'https://www.ptt.cc/bbs/'
def crawl_board(board_name, board_page):
    url = base_url + board_name + '/index' + str(board_page) + '.html'
    # res = requests.get(url=url)
    res = requests.get(url=url, cookies={'over18': '1'})

    print(res)
    # print(res.text)
    save_html(res.text)

crawl_board('Gossiping', 36999)

<Response [200]>


### 爬取內文

我們已經可以自由的選擇版面和頁數了，這時候如果我們想要爬內文呢?
例如:
https://www.ptt.cc/bbs/Gossiping/M.1553696869.A.7F7.html

In [7]:
def crawl_page(page_url):
    # res = requests.get(url=url)
    res = requests.get(url=page_url, cookies={'over18': '1'})

    print(res)
    # print(res.text)
    save_html(res.text)

crawl_page('https://www.ptt.cc/bbs/Gossiping/M.1553696869.A.7F7.html')

<Response [200]>


# 自動化 (讓 Python 像人一樣操縱瀏覽器)

現在都還是我們自己去輸入要爬的頁面，需要耗費人工的力氣。

## 解析 html

使用 Beautiful Soup 解析爬取的網頁，可以透過這個找到 **按鈕連結** 和 **網站的內容**

```python
soup = BeautifulSoup(res.text, 'html.parser')
```

In [8]:
from bs4 import BeautifulSoup
print(BeautifulSoup)

<class 'bs4.BeautifulSoup'>


In [9]:
base_url = 'https://www.ptt.cc/bbs/'
def crawl_board(board_name, board_page):
    url = base_url + board_name + '/index' + str(board_page) + '.html'

    res = requests.get(url=url, cookies={'over18': '1'})
    print('正在爬取', url)
    save_html(res.text)
    
    soup = BeautifulSoup(res.text, 'html.parser')
    print(soup.title)
    
    page_titles = soup.find_all(class_='title')
    print(page_titles)
    print(len(page_titles))
    
crawl_board('Gossiping', 36999)

正在爬取 https://www.ptt.cc/bbs/Gossiping/index36999.html
<title>看板 Gossiping 文章列表 - 批踢踢實業坊</title>
[<div class="title">
<a href="/bbs/Gossiping/M.1551924564.A.E16.html">[問卦] 蔡政府不是說超收稅金要發錢補助窮人/?</a>
</div>, <div class="title">
<a href="/bbs/Gossiping/M.1551924577.A.636.html">[問卦] 喝葷茶是什麼？</a>
</div>, <div class="title">
<a href="/bbs/Gossiping/M.1551924643.A.86B.html">[新聞] 老爺爺買下波音727！建成「巨型精密豪宅</a>
</div>, <div class="title">
<a href="/bbs/Gossiping/M.1551924769.A.1C4.html">[新聞] 三立、壹電視新聞台「非洲豬瘟」內容遭NC</a>
</div>, <div class="title">
<a href="/bbs/Gossiping/M.1551924775.A.CE4.html">Re: [問卦] 尼特羅會長 VS 全部幻影旅團，哪邊會贏?</a>
</div>, <div class="title">
<a href="/bbs/Gossiping/M.1551924801.A.E1E.html">[新聞] 王金平今宣布參選2020總統</a>
</div>, <div class="title">
<a href="/bbs/Gossiping/M.1551924875.A.B9C.html">Re: [問卦] 認真玩相機跟音響哪個比較貴？</a>
</div>, <div class="title">
<a href="/bbs/Gossiping/M.1551924911.A.2CE.html">Re: [新聞] 柯粉快護駕! (訪美行程比一比</a>
</div>, <div class="title">
<a href="/bbs/Gossiping/M.1551924932.A.B0A.ht

## 進一步讀取內部資訊

HTML是一個格式，資訊都用`<>`包起來，我們可以進一步解析連結網址與標題

In [10]:
base_url = 'https://www.ptt.cc/bbs/'
def crawl_board(board_name, board_page):
    url = base_url + board_name + '/index' + str(board_page) + '.html'

    res = requests.get(url=url, cookies={'over18': '1'})
    print('正在爬取', url)
    save_html(res.text)
    
    soup = BeautifulSoup(res.text, 'html.parser')
    sub_pages = soup.find_all('div', class_='r-ent')
    sub_pages = sub_pages[:5]
    
    for sub_page in sub_pages:
        
        # 找推數
        push_info = sub_page.find(class_='nrec').find('span')
        if push_info != None:
            num_push = push_info.text
        else:
            num_push = 0
            
        # 找連結資訊
        link = sub_page.find('a')
        link_url = link['href']
        link_title = link.text
        
        # 找日期
        date = sub_page.find(class_='date').text
        
        print(link_url, link_title, num_push, date)

crawl_board('Gossiping', 36999)

正在爬取 https://www.ptt.cc/bbs/Gossiping/index36999.html
/bbs/Gossiping/M.1551924564.A.E16.html [問卦] 蔡政府不是說超收稅金要發錢補助窮人/? 0  3/07
/bbs/Gossiping/M.1551924577.A.636.html [問卦] 喝葷茶是什麼？ 3  3/07
/bbs/Gossiping/M.1551924643.A.86B.html [新聞] 老爺爺買下波音727！建成「巨型精密豪宅 13  3/07
/bbs/Gossiping/M.1551924769.A.1C4.html [新聞] 三立、壹電視新聞台「非洲豬瘟」內容遭NC 0  3/07
/bbs/Gossiping/M.1551924775.A.CE4.html Re: [問卦] 尼特羅會長 VS 全部幻影旅團，哪邊會贏? 7  3/07


## 解析內頁

我們把類似的作法套用在內頁，得到內頁的標題與內文

In [11]:
def filter_content(content):
    content = content.replace('\n', '')
    match = re.search(r'^(.*?)※\ 發信站:\ 批踢踢實業坊\(ptt\.cc\)', content, re.MULTILINE)
    if match:
        content = match.group(1)
    content = re.sub(r'(<.+?>).*?', '' ,content)
    content = re.sub(r'.*?(</.+?>)', '' ,content)
    print(content[:100])
    
    return content
    # print(content)

In [12]:
def parse_page(soup):
    meta_data = soup.find_all(class_='article-metaline')
    # 回傳一個 list，分別是: 作者、標題、時間
    # print(meta_data)
    
    author_data = meta_data[0].find(class_='article-meta-value').text
    title = meta_data[1].find(class_='article-meta-value').text
    date = meta_data[2].find(class_='article-meta-value').text
    
    author = author_data.split(' ')[0]
    sign = author_data.split(' ')[1]
    
    print(author)
    print(sign)
    print(title)
    print(date)
    
    # 解析內文
    content = soup.find(id='main-content').contents[4:]
    content = ''.join([str(c) for c in content])
    content = filter_content(content)
    

def crawl_page(page_url):
    # res = requests.get(url=url)
    res = requests.get(url=page_url, cookies={'over18': '1'})
    print('正在爬取', page_url)

    soup = BeautifulSoup(res.text, 'html.parser')
    parse_page(soup)

crawl_page('https://www.ptt.cc/bbs/Gossiping/M.1553696869.A.7F7.html')
crawl_page('https://www.ptt.cc/bbs/Gamesale/M.1553747800.A.799.html')

正在爬取 https://www.ptt.cc/bbs/Gossiping/M.1553696869.A.7F7.html
b10485762000
(台北寂寞部屋)
[新聞] 88歲老榮民節儉度日 大愛賣屋捐700萬
Wed Mar 27 22:27:46 2019
88歲老榮民節儉度日 大愛賣屋捐700萬中央社 - 2019-03-27 10:13:56（中央社記者魯鋼駿、管瑞平新竹市27日電）新竹榮民之家88歲榮民劉明芝，平時相當節省，一條褲子穿了20多年仍不
正在爬取 https://www.ptt.cc/bbs/Gamesale/M.1553747800.A.799.html
jindulan
(jindulan)
[PS4] 徵 全境封鎖2
Thu Mar 28 12:36:37 2019
★【物品名稱】：徵全境封鎖2★【遊戲分級】：限制級★【語系版本】：中文★【徵 求 價】：1400★【交易方式】：面交　【保存狀況】：全新或二手無刮　【地    區】：捷運板橋到西門均可　【附　　註】：


現在已經可以爬取「版」頁面的資訊與內頁的資訊了，是時候將他們兩個串在一起，達到自動爬取內頁的功能，整格流程像是：

爬某個版的某一頁 -> 讀取內頁連結 -> 爬裡面所有的內頁

In [13]:
import time
base_url = 'https://www.ptt.cc/bbs/'
def crawl_board(board_name, board_page):
    url = base_url + board_name + '/index' + str(board_page) + '.html'

    res = requests.get(url=url, cookies={'over18': '1'})
    print('正在爬取', url)
    save_html(res.text)
    
    soup = BeautifulSoup(res.text, 'html.parser')
    sub_pages = soup.find_all('div', class_='r-ent')
#     sub_pages = sub_pages[:5]
    
    for sub_page in sub_pages:
        
        # 找推數
        push_info = sub_page.find(class_='nrec').find('span')
        if push_info != None:
            num_push = push_info.text
        else:
            num_push = 0
            
        # 找連結資訊
        link = sub_page.find('a')
        link_url = link['href']
        link_title = link.text
        
        # 找日期
        date = sub_page.find(class_='date').text
                
        print(link_url, link_title, num_push, date)
        
        sub_page_base_url = 'https://www.ptt.cc'
        time.sleep(1)
        crawl_page(sub_page_base_url + link_url)
        
        print('')
        print('-------------------')
        print('')
        
crawl_board('Gossiping', 36999)

正在爬取 https://www.ptt.cc/bbs/Gossiping/index36999.html
/bbs/Gossiping/M.1551924564.A.E16.html [問卦] 蔡政府不是說超收稅金要發錢補助窮人/? 0  3/07
正在爬取 https://www.ptt.cc/bbs/Gossiping/M.1551924564.A.E16.html
onetwogo
(中國優質新聞搬運工)
[問卦] 蔡政府不是說超收稅金要發錢補助窮人/?
Thu Mar  7 10:09:21 2019
 大家好 我是窮人 之前政府不是說稅金收了好多， 爽收臺灣人民血汗錢收了這麼多 不是說要發錢補助窮人 怎麼沒消息了?? 怎麼不發了呢?? 這樣不行喔!!!言而無信!!你會連任不了喔!!! 哦!!本來就

-------------------

/bbs/Gossiping/M.1551924577.A.636.html [問卦] 喝葷茶是什麼？ 3  3/07
正在爬取 https://www.ptt.cc/bbs/Gossiping/M.1551924577.A.636.html
Raphael666
(Raphael)
[問卦] 喝葷茶是什麼？
Thu Mar  7 10:09:35 2019
如題，小弟來到一間茶館，這裡的規矩很奇怪，要自己帶茶葉去泡，喝茶還配肉，不怕拉肚子嗎？ 這裡的人都叫喝葷茶，到底是什麼東西，有八卦嗎？ss-----Sent from JPTT on my Sony 

-------------------

/bbs/Gossiping/M.1551924643.A.86B.html [新聞] 老爺爺買下波音727！建成「巨型精密豪宅 13  3/07
正在爬取 https://www.ptt.cc/bbs/Gossiping/M.1551924643.A.86B.html
yafabo
(認真的男人最帥)
[新聞] 老爺爺買下波音727！建成「巨型精密豪宅
Thu Mar  7 10:10:36 2019
三立老爺爺買下波音727！建成「巨型精密豪宅」住森林20年國際中心／綜合報導世界之大無奇不有，就連居住的地方都可以別出心裁的設計。美國奧勒岡州69歲的老爺爺坎貝爾（Bruce Cambell）因為從年

--------

## 將結果傳出來

目前為止做的是「拿到結果就印出來」，但這不利於之後做分析或儲存，所以我們要在函式中回傳爬下來的資料，也就是加上`return`回傳資料


### 儲存格式

對一個內頁的資訊，我們用一個`dictionary`去存，大致上長這樣
```python
{
    'author': 作者,
    'title': 標題,
    'date': 時間,
    'content': 內容
}
```
要儲存多個內頁的資訊，外面再用一個 `list` 去包起來，會變成像這樣
```python
[
    {內頁1資訊}, {內頁2資訊}, {內頁3資訊} ...
]
```

In [15]:
# 內頁

def filter_content(content):
    content = content.replace('\n', '')
    match = re.search(r'^(.*?)※\ 發信站:\ 批踢踢實業坊\(ptt\.cc\)', content, re.MULTILINE)
    if match:
        content = match.group(1)
    content = re.sub(r'(<.+?>).*?', '' ,content)
    content = re.sub(r'.*?(</.+?>)', '' ,content)
    
    return content

def parse_page(soup):
    meta_data = soup.find_all(class_='article-metaline')
    author = meta_data[0].find(class_='article-meta-value').text
    title = meta_data[1].find(class_='article-meta-value').text
    date = meta_data[2].find(class_='article-meta-value').text
    
    # 解析內文
    content = soup.find(id='main-content').contents[4:]
    content = ''.join([str(c) for c in content])
    content = filter_content(content)
    
    data = {'author': author, 'title': title, 'date': date, 'content': content }
    return data
    

def crawl_page(page_url):
    res = requests.get(url=page_url, cookies={'over18': '1'})
    soup = BeautifulSoup(res.text, 'html.parser')
    data = parse_page(soup)
    
    return data

sub_page_data = crawl_page('https://www.ptt.cc/bbs/Gossiping/M.1553696869.A.7F7.html')

print(sub_page_data)
# 比較漂亮的打印方法
# import pprint
# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(sub_page_data)


{'author': 'b10485762000 (台北寂寞部屋)', 'title': '[新聞] 88歲老榮民節儉度日 大愛賣屋捐700萬', 'date': 'Wed Mar 27 22:27:46 2019', 'content': '88歲老榮民節儉度日 大愛賣屋捐700萬中央社 - 2019-03-27 10:13:56（中央社記者魯鋼駿、管瑞平新竹市27日電）新竹榮民之家88歲榮民劉明芝，平時相當節省，一條褲子穿了20多年仍不捨丟棄，但他去年卻將賣屋所得700多萬全數捐給新竹榮民之家，善心義舉令人感佩。劉明芝18歲時隨軍隊來台，民國45年上兵退役後擔任清潔隊員，一做就是34年，靠著微薄就養金和清潔員收入度日，節儉的他常一顆饅頭配白開水打發一餐。民國71年時，劉明芝用辛苦存下的新台幣80萬元，在台北市萬華地區買了間公寓，在那裡度過30多年歲月，直到去年入住新竹榮民之家。新竹榮民之家主任陳桂美說，劉明芝的雙親很早就過世，因此他來台70年未曾返鄉探親，甚至沒有出國、離開過台灣這片土地，加上因未婚、沒有子女，一直獨居在萬華小公寓，基本上未享受過親情與物質生活。陳桂美表示，去年9月劉明芝入住新竹榮民之家後，感受到「家」的溫暖，決定把房子賣掉，並將賣屋所得700多萬元全數捐給新竹榮家及榮民榮眷基金會，盼能改善榮家的生活設施。工作人員曾多次勸劉明芝不要捐這麼多錢，但他卻說：「錢財乃身外之物，生不帶來，死不帶去」他想盡些力量回饋照顧他一生的國家。新竹榮民之家工作人員分享，劉明芝相當節省、惜物，至今仍穿著20多年前清潔隊公發的工作褲，即使已多處破洞仍不捨丟棄，常向工作人員借針線縫補，眾人勸他買新褲子，但他總是回應「還可以穿」，儉樸的個性令人敬佩。劉明芝儉樸度日，卻將一生心血捐出，甚至不留分文在身，為感謝他的善心義舉，退輔會近日頒發「榮光專業一等獎章」以表彰他的善行義舉。（編輯：李亨山）10803274.完整新聞連結:https://www.nownews.com/news/20190327/3292747/ --'}


In [16]:
# 版面頁
base_url = 'https://www.ptt.cc/bbs/'
def crawl_board(board_name, board_page):
    url = base_url + board_name + '/index' + str(board_page) + '.html'
    res = requests.get(url=url, cookies={'over18': '1'})
    
    soup = BeautifulSoup(res.text, 'html.parser')
    sub_pages = soup.find_all('div', class_='r-ent')
    
    # 用一個 list 把內頁資訊存起來
    sub_page_data_list = []
    
    for sub_page in sub_pages:
        
        # 找推數
        push_info = sub_page.find(class_='nrec').find('span')
        if push_info != None:
            num_push = push_info.text
        else:
            num_push = 0
            
        # 找連結資訊
        link = sub_page.find('a')
        link_url = link['href']
        link_title = link.text
        
        # 找日期
        date = sub_page.find(class_='date').text
                
        sub_page_base_url = 'https://www.ptt.cc'
        sub_page_data = crawl_page(sub_page_base_url + link_url) # 注意，把回傳直接起來
        
        # 補上 link 的資訊
        sub_page_data['link'] = sub_page_base_url + link_url
        sub_page_data['num_push'] = num_push
        
        # 在準備好的 list 上加上這個資訊
        sub_page_data_list.append(sub_page_data)
    
    return sub_page_data_list

page_data = crawl_board('Gossiping', 36999)
print(page_data[:5])

[{'author': 'onetwogo (中國優質新聞搬運工)', 'title': '[問卦] 蔡政府不是說超收稅金要發錢補助窮人/?', 'date': 'Thu Mar  7 10:09:21 2019', 'content': ' 大家好 我是窮人 之前政府不是說稅金收了好多， 爽收臺灣人民血汗錢收了這麼多 不是說要發錢補助窮人 怎麼沒消息了?? 怎麼不發了呢?? 這樣不行喔!!!言而無信!!你會連任不了喔!!! 哦!!本來就選不上是嘛?? 民調都最低 所以我是想要問一下啦 政府怎麼不發錢給窮人了呢?? 嘻嘻 ps:台灣窮人素質都很差不用可憐他們，他們死好  我算是特例啦，素質好卻要變窮人!!沒辦法太倒楣了...  科科--', 'link': 'https://www.ptt.cc/bbs/Gossiping/M.1551924564.A.E16.html', 'num_push': 0}, {'author': 'Raphael666 (Raphael)', 'title': '[問卦] 喝葷茶是什麼？', 'date': 'Thu Mar  7 10:09:35 2019', 'content': '如題，小弟來到一間茶館，這裡的規矩很奇怪，要自己帶茶葉去泡，喝茶還配肉，不怕拉肚子嗎？ 這裡的人都叫喝葷茶，到底是什麼東西，有八卦嗎？ss-----Sent from JPTT on my Sony G8142.--', 'link': 'https://www.ptt.cc/bbs/Gossiping/M.1551924577.A.636.html', 'num_push': '3'}, {'author': 'yafabo (認真的男人最帥)', 'title': '[新聞] 老爺爺買下波音727！建成「巨型精密豪宅', 'date': 'Thu Mar  7 10:10:36 2019', 'content': '三立老爺爺買下波音727！建成「巨型精密豪宅」住森林20年國際中心／綜合報導世界之大無奇不有，就連居住的地方都可以別出心裁的設計。美國奧勒岡州69歲的老爺爺坎貝爾（Bruce Cambell）因為從年輕時就相當熱愛飛機，20年前向航空公司以10萬美金左右（約新台幣310萬元）買下一架波音727客機，並且搬進森林中，就這樣住在飛機裡長達2

## 處理多頁的資訊
剛剛我們都以讀八卦版的第36999頁為例，現在我們設定一個頁數範圍，讀取每一頁，然後把資訊併在一起

### 補充
這裡的`try`是用來避免我們在爬取資料的時候遇到沒考慮到的錯誤，意思是，如果爬取這頁時遇到問題，就跳過它，用來節省一些麻煩。

In [17]:
def crawl_board_with_range(start, end):
    
    board_data = []
    for page_id in range(start, end):
        print('爬取第', page_id, '頁')
        try:
            page_data = crawl_board('Gossiping', page_id)
            board_data += page_data
        except:
            print('遇到問題，跳過此頁')
            pass
            
    return board_data

crawl_board_with_range(36000, 36002) # 小心值不要設太大，會跑很久

爬取第 36000 頁
爬取第 36001 頁


[{'author': 'b10485762000 (台北寂寞部屋)',
  'title': '[新聞] 貧困母女被害人搭6hr公車出庭\u30002司法官爭',
  'date': 'Mon Feb 25 10:05:28 2019',
  'content': '貧困母女被害人搭6hr公車出庭\u30002司法官爭掏腰包獲讚佛心出版時間：2019/02/24 19:00  蘋果日報2月14日情人節當天，台北地院審理一起性侵案，法庭內出現難得溫馨場面，一位女法官發現家境不好的被害女大生與母親，大老遠從台東搭6小時車到法院，最後還慈悲原諒被告，卻因被害人依規定無證人旅費，正打算自掏腰包補貼母女車資，公訴檢察官竟愛心不落人後，搶先掏出千元，令這對母女十分感念，激動地連聲鞠躬道謝。事後法官堅持給錢，檢察官才同意各出一半，記者昨找到這位佛心法官，她低調地說：「被害人母女真的很弱勢、單純，只是盡一點心意而已。」檢察官則回應：「共襄盛舉啦！」法庭暖流消息傳出後，這位佛心法官的審判長在臉書貼文分享2位司法官的善舉，許多臉友紛紛轉載，並高度讚許這2位司法官愛心不落人後的事蹟。審判長臉書貼文寫道「...最近，台北地院法官開準備程序庭時，依法傳喚被告、未成年被害人到庭。被告曾是被害人阿姨的同居人，自始坦承犯行，被害人也原諒被告了，但被告因為資力困難，無法賠償被害人。如何判刑，只能留待合議庭他日開庭審結。被害人與其母親接到傳票時，不知可以不用到庭（可以具狀陳述意見即可），因此即便家中經濟狀況欠佳，母女三人仍坐了數小時的大眾運輸工具，從台東北上台北。」「法官開庭知悉詳情後，原本打算發證人旅費給被害人母女，但因為傳票是以被害人身分傳喚，依法不得發給證人旅費。法官一時無計，遂打算自掏腰包，算是貼補交通費用。一掏口袋後，才發現未帶現金下來開庭。臨時代班的公訴檢察官見狀，從口袋掏出千元鈔給被害人母女，被害人母女離開時，謝聲連連！法官中場休息去拿錢後，有意償付檢察官出的錢，檢察官拒絕收受。一番推辭後，兩人折衷各出一半。人間處處有溫情，佛心法官、檢察官展現了人間情！」記者多方打聽，好不容易找到這位佛心女法官，她堅持不願曝光，還謙虛地說：「這種事應該很多，不是只有我會做。」這位女法官表示，當天開庭前得知被害人住台東，社工通知不會出庭，因此沒有安排有隔離訊問室的法庭，沒想到開庭時，突然接獲通知被害人與

## 完成了，然後呢？
我們成功的把ptt某個版的資訊爬下來了，然後可以做些什麼？

1. 把 `board_data` 存下來
2. 對 `board_data` 分析或是搜尋

## 儲存方法
儲存資料的方法有很多種，我們這裡採取比較簡單的方式，利用`pickle`直接把 Python 的陣列存成一個檔案。

這裡提供了儲存與讀取的寫法，如果已經爬過並且儲存起來，就可以只要讀取就好，把其他的部分註解掉。

In [18]:
import pickle

# 注意：跑[35000, 35100]有ㄧ100頁，約要一個小時以上。
board_data = crawl_board_with_range(35000, 35100)

# 將爬下來的資料 list 存成 board_data.pkl
with open('board_data.pkl', 'wb') as output:
    pickle.dump(board_data, output)

# 將 board_data.pkl 讀出來到 loaded_board_data 這個變數
with open('board_data.pkl', 'rb') as input:
    loaded_board_data = pickle.load(input)
    print(len(loaded_board_data))

1980


## 簡單的應用
在所有的文章中搜尋關鍵字，並且印出搜尋到的文章的相關資訊。

小提醒，我們儲存的資料長這樣
```python
[
    {內頁1資訊}, {內頁2資訊}, {內頁3資訊} ...
]
```

In [24]:
# board_data = crawl_board_with_range(35000, 35500)

def search_word_in_data(data, keyword):
    for page_data in data:
        if keyword in page_data['content']:
            print('在', page_data['title'], '找到', keyword, '關鍵字')            
            print('連結:', page_data['link'])

# 直接把我們儲存的資料讀出來的那份丟進去
search_word_in_data(loaded_board_data, '急診')

在 [新聞] 小六轉學生淪人肉沙包　同學陰狠四招害렱 找到 急診 關鍵字
連結: https://www.ptt.cc/bbs/Gossiping/M.1550291348.A.6E3.html
在 Re: [問卦] 台女到底討不討厭小馮這樣劈腿的行為？ 找到 急診 關鍵字
連結: https://www.ptt.cc/bbs/Gossiping/M.1550298711.A.6F2.html
在 [問卦] 大夜急診護理工作妹子 找到 急診 關鍵字
連結: https://www.ptt.cc/bbs/Gossiping/M.1550312538.A.843.html
在 [新聞] 段宜康料理大螃蟹　慘！秒遭夾傷拇指就醫 找到 急診 關鍵字
連結: https://www.ptt.cc/bbs/Gossiping/M.1550331900.A.EB5.html
在 [問卦] 有原住民醫師不是加分仔的嗎？ 找到 急診 關鍵字
連結: https://www.ptt.cc/bbs/Gossiping/M.1550347218.A.95F.html
在 [問卦] 醫生是不是10年內會變低薪族群？ 找到 急診 關鍵字
連結: https://www.ptt.cc/bbs/Gossiping/M.1550361516.A.2D2.html
在 Re: [問卦] 醫生是不是10年內會變低薪族群？ 找到 急診 關鍵字
連結: https://www.ptt.cc/bbs/Gossiping/M.1550366373.A.2CA.html
在 Re: [問卦] 醫生是不是10年內會變低薪族群？ 找到 急診 關鍵字
連結: https://www.ptt.cc/bbs/Gossiping/M.1550371046.A.963.html
在 [新聞] 勇！半馬女跑者完賽前遭大狗猛咬 浴血衝 找到 急診 關鍵字
連結: https://www.ptt.cc/bbs/Gossiping/M.1550373327.A.471.html
